In [1]:
#!pip install tensorflow
#!pip install keras

In [2]:
import pandas as pd
import numpy as np

# load dataset
dataset = pd.read_csv("diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset.iloc[:,0:8]
Y = dataset.iloc[:,8]


In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

from sklearn.model_selection import GridSearchCV

In [4]:
def create_model(neurons=1):
	# create model
	model = Sequential()
	model.add(Dense(neurons, input_dim=8, kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(4)))
	model.add(Dropout(0.2))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid')) # output layer with 1 neuron 
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

In [5]:
# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)


In [6]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.718750 using {'neurons': 10}
0.708333 (0.020505) with: {'neurons': 1}
0.707031 (0.013902) with: {'neurons': 5}
0.718750 (0.027805) with: {'neurons': 10}
0.710938 (0.030425) with: {'neurons': 15}
0.717448 (0.025780) with: {'neurons': 20}
0.707031 (0.011500) with: {'neurons': 25}
0.704427 (0.022628) with: {'neurons': 30}
